## Model Evaluation 

In [1]:
# import dependencies 
import math 
import numpy as np
from sklearn.metrics.cluster import rand_score
from itertools import combinations

In [2]:
# Define input variables  
# group_dist = [[0], [1], [2], [3]]
# group_dist = [[0, 1], [2, 3]]
group_dist = [[0, 1], [0, 1]] # list must be ordered in ascending order   
vote_dist = [9, 9]
n_agents = 2

In [3]:
def group_memberships(groups, n_agents):
    """
    Define group memberships for each participant.
    :param: groups (list of lists): a list denotes the group and contains its members. 
    :param: votes (list): number of participant's votes for a given project proposal.
    :returns (list of lists): retunrs a list of group membersips for each participant.  
    """
    # let each entry in votes define a unique identifyer for an agent 
    group_memberships = [[] for _ in range(n_agents)]

    # add group memberships of agent i as a list to each entry identifying an agent 
    for i in range(len(groups)):
        for j in groups[i]:
            group_memberships[j].append(i)
        
    return group_memberships

In [4]:
result_group_memberships = group_memberships(groups=group_dist, n_agents=n_agents)
# print(result_group_memberships)

In [5]:
def common_group(first_agent, second_agent, memberships_group):
    """
    Define an identifyer indicating whether two participants share the same group or whehter any
    other member of the group of the second agent shares a group with the first agent. 
    :param: agent_i denotes a participant not equal to a participant called agent_j. 
    :param: agent_j denotes a participant not equal to a participant called agent_i.
    :returns (bool): returns true if two participants share a common group and false otherwise.  
    """
    # debug 
    # print(f"agent_i: {first_agent}, agent_j: {second_agent}")
    # print(f"group_memberships[agent_i]: {memberships_group[first_agent]}, type: {type(memberships_group[first_agent])}")
    # print(f"group_memberships[agent_j]: {memberships_group[second_agent]}, type: {type(memberships_group[second_agent])}")
    
    common_group = any(group in memberships_group[second_agent] for group in memberships_group[first_agent])

    return common_group

In [6]:
# result_commom_group = common_group(0,1, result_group_memberships)
# print(result_commom_group)

In [7]:
def K(agent_i, group, votes, memberships_group):
    """
    Define the weighting function that attenuates the votes of agent i given different group memberships.  
    :param: agent_i denotes a participant not equal to a participant called agent_j.
    :param: (integer): group denotes the group number.
    :returns: attenuated number of votes for a given project. 
    """
    if agent_i in group or any(common_group(agent_i, j, memberships_group) for j in group):
        return math.sqrt(votes[agent_i])
    else:
        return votes[agent_i]


In [8]:
# result_K = K(0, group_dist[1], vote_dist, result_group_memberships)
# print(result_K)

In [9]:
def first_term(groups, votes, memberships_group):
    """
    Calculate the first term of the formula for "connection-oriented cluster match" by individual and in aggregate.
    :param: groups (list of lists): a list denotes the group and contains its members.
    :param: votes (list): number of participant's votes for a given project proposal.
    :param: group_membership: defines group memberships for each participant.
    """ 
    weighted_votes_groups = {}
    for group_num, group in enumerate(groups):
        
        weighted_votes_individual = {}
        for agent_i in group:

            votes_i = votes[agent_i]
            num_groups_i = len(memberships_group[agent_i])
            vote_weight_i = votes_i/num_groups_i
            
            agent_key = f"agent_{agent_i}"
            weighted_votes_individual[agent_key] = {
                    'votes': votes_i,
                    'num_groups': num_groups_i,
                    'vote_weight': vote_weight_i
                }

        group_key = f"group_{group_num}"
        weighted_votes_groups[group_key] = weighted_votes_individual 
    
    aggregate_votes_weight = sum(agent_data["vote_weight"] for group_data in weighted_votes_groups.values() for agent_data in group_data.values())
    
    return weighted_votes_groups, aggregate_votes_weight

In [10]:
result_groups, result_aggregate_weight = first_term(group_dist, vote_dist, result_group_memberships)
# print(result_groups)
# print(result_aggregate_weight)

In [11]:
def interaction_terms(groups, memberships_group, votes):
    """
    Calculate the interaction terms of the formula for "connection-oriented cluster match" by individual and in aggregate.
    :param: groups (list of lists): a list denotes the group and contains its members.
    :param: votes (list): number of participant's votes for a given project proposal.
    :param: group_membership: defines group memberships for each participant.
    """ 
    interaction_terms_individual = {}
    for group_num, group in enumerate(groups):
        interaction_terms_group = {}

        for other_group_num, other_group in enumerate(groups):
            if group_num == other_group_num: # index comparision allows to compare groups that have the same composition of participants 
                continue
            
            result_agent_i = {} 
            for agent_i in group: 
                vote_attenuation_i = K(agent_i, other_group, votes, memberships_group) 
                num_groups_i = len(memberships_group[agent_i])
                vote_weight_i = vote_attenuation_i/num_groups_i

                agent_i_key = f"agent_i_{agent_i}"
                result_agent_i[agent_i_key] = {
                    'vote_attenuation_i': vote_attenuation_i,
                    'num_groups_i': num_groups_i,
                    'vote_weight_i': vote_weight_i
                }
                
            result_agent_j = {} 
            for agent_j in other_group:
                vote_attenuation_j = K(agent_j, group, votes, memberships_group)
                num_groups_j = len(memberships_group[agent_j])
                vote_weight_j = vote_attenuation_j/num_groups_j

                agent_j_key = f"agent_j_{agent_j}"
                result_agent_j[agent_j_key] = {
                    'vote_attenuation_j': vote_attenuation_j,
                    'num_groups_j': num_groups_j,
                    'vote_weight_j': vote_weight_j
                }

            # Caluclation of individual internaction terms 
            sum_vote_weight_i = sum(value['vote_weight_i'] for value in result_agent_i.values())
            sum_vote_weight_j = sum(value['vote_weight_j'] for value in result_agent_j.values()) 
            sqrt_sum_vote_weight_i = math.sqrt(sum_vote_weight_i)
            sqrt_sum_vote_weight_j = math.sqrt(sum_vote_weight_j)

            #interaction_term1 = math.sqrt(sum(K(agent_i, other_group) / len(group_memberships[agent_i]) for agent_i in group))
            #interaction_term2 = math.sqrt(sum(K(agent_j, group) / len(group_memberships[agent_j]) for agent_j in other_group))

            other_group_key = f"other_group_{other_group_num}"
            interaction_terms_group[other_group_key] = {
                'components_int_term1': result_agent_i,
                'components_int_term2': result_agent_j,
                'sum_vote_weight_i': sum_vote_weight_i,
                'sqrt_sum_vote_weight_i': sqrt_sum_vote_weight_i,
                'sum_vote_weight_j': sum_vote_weight_j,
                'sqrt_sum_vote_weight_j': sqrt_sum_vote_weight_j,
                'multiplied_interaction': sqrt_sum_vote_weight_i*sqrt_sum_vote_weight_j 
            }

        group_key = f"group_{group_num}"
        interaction_terms_individual[group_key] = interaction_terms_group
    
    aggregated_interaction_terms = sum(value['multiplied_interaction'] for group_data in interaction_terms_individual.values() for value in group_data.values())
    
    return interaction_terms_individual, aggregated_interaction_terms

In [12]:
result_individual, result_aggregated = interaction_terms(group_dist, result_group_memberships, vote_dist)
# print(result_individual)
# print(result_aggregated)

In [13]:
def interaction_terms_new(groups, memberships_group, votes):
    """
    Calculate the interaction terms of the formula for "connection-oriented cluster match" by individual and in aggregate.
    :param: groups (list of lists): a list denotes the group and contains its members.
    :param: votes (list): number of participant's votes for a given project proposal.
    :param: group_membership: defines group memberships for each participant.
    """ 
    interaction_terms_individual = {}
    for group in groups:
        interaction_terms_group = {}

        for other_group in groups:
            if group == other_group: 
                continue
            
            result_agent_i = {} 
            for agent_i in group: 
                vote_attenuation_i = K(agent_i, other_group, votes, memberships_group) 
                num_groups_i = len(memberships_group[agent_i])
                vote_weight_i = vote_attenuation_i/num_groups_i

                agent_i_key = f"agent_i_{agent_i}"
                result_agent_i[agent_i_key] = {
                    'vote_attenuation_i': vote_attenuation_i,
                    'num_groups_i': num_groups_i,
                    'vote_weight_i': vote_weight_i
                }
                
            result_agent_j = {} 
            for agent_j in other_group:
                vote_attenuation_j = K(agent_j, group, votes, memberships_group)
                num_groups_j = len(memberships_group[agent_j])
                vote_weight_j = vote_attenuation_j/num_groups_j

                agent_j_key = f"agent_j_{agent_j}"
                result_agent_j[agent_j_key] = {
                    'vote_attenuation_j': vote_attenuation_j,
                    'num_groups_j': num_groups_j,
                    'vote_weight_j': vote_weight_j
                }

            # Caluclation of individual internaction terms 
            sum_vote_weight_i = sum(value['vote_weight_i'] for value in result_agent_i.values())
            sum_vote_weight_j = sum(value['vote_weight_j'] for value in result_agent_j.values()) 
            sqrt_sum_vote_weight_i = math.sqrt(sum_vote_weight_i)
            sqrt_sum_vote_weight_j = math.sqrt(sum_vote_weight_j)

            #interaction_term1 = math.sqrt(sum(K(agent_i, other_group) / len(group_memberships[agent_i]) for agent_i in group))
            #interaction_term2 = math.sqrt(sum(K(agent_j, group) / len(group_memberships[agent_j]) for agent_j in other_group))

            other_group_key = f"other_group_{other_group}"
            interaction_terms_group[other_group_key] = {
                'components_int_term1': result_agent_i,
                'components_int_term2': result_agent_j,
                'sum_vote_weight_i': sum_vote_weight_i,
                'sqrt_sum_vote_weight_i': sqrt_sum_vote_weight_i,
                'sum_vote_weight_j': sum_vote_weight_j,
                'sqrt_sum_vote_weight_j': sqrt_sum_vote_weight_j,
                'multiplied_interaction': sqrt_sum_vote_weight_i*sqrt_sum_vote_weight_j 
            }

        group_key = f"group_{group}"
        interaction_terms_individual[group_key] = interaction_terms_group
    
    aggregated_interaction_terms = sum(value['multiplied_interaction'] for group_data in interaction_terms_individual.values() for value in group_data.values())
    
    return interaction_terms_individual, aggregated_interaction_terms

In [14]:
result_individual_new, result_aggregated_new = interaction_terms_new(group_dist, result_group_memberships, vote_dist)
# print(result_individual_new)
# print(result_aggregated_new)

In [15]:
def connection_oriented_cluster_match(term_1, interaction_terms):
    """
    This function calculates the overall plurality score of a project by 
    transforming the quadratic finance formula to quadratic voting 
    """
    weighted_proposal_votes = term_1 + interaction_terms 
    sqrt_weighted_proposal_votes = math.sqrt(weighted_proposal_votes)

    return sqrt_weighted_proposal_votes
    

In [16]:
plurality_score = connection_oriented_cluster_match(result_aggregate_weight, result_aggregated_new)
# print(plurality_score)

### Quadratic Voting 

In [17]:
def quadratic_voting(votes): 
    """
    :param: vote_dist (list): number of participant's votes for a given project proposal.
    :returns (dict): a dictionary containing quadratic votes for each agent i and the sum of quadratic vote.  
    """
    quadratic_votes_dict = {}
    for agent_i in range(len(votes)):
        quadratic_votes_angent_i = math.sqrt(votes[agent_i])
        quadratic_votes_dict[agent_i] = quadratic_votes_angent_i

    sum_quadratic_votes = sum(quadratic_votes_dict.values())
    
    return quadratic_votes_dict, sum_quadratic_votes

In [18]:
result_quadratic_voting, result_sum_quadratic_votes = quadratic_voting(vote_dist)
# print(result_quadratic_voting)
# print(result_sum_quadratic_votes)

### Rand Index 

In [19]:
def rand_index(partitions):
    """
    Compute the Rand Index for a set of partitions.
    :param partitions: List of lists, where each list represents a partition.
    :return: Rand Index value.
    """
    true_labels = []
    all_labels = []

    for i, partition in enumerate(partitions):
        for group in partition:
            if isinstance(group, list):
                true_labels.extend([i] * len(group))
                all_labels.extend(group)
            else: # handle groups of size 1 
                true_labels.extend([i] * 1)  
                all_labels.extend([group])

    return rand_score(true_labels, all_labels)

### Jaccard Index 

In [20]:
def jaccard_similarity(set1, set2):
    """
    Compute the Jaccard similarity between two sets.
    :param set1: First set.
    :param set2: Second set.
    :return: Jaccard similarity value.
    """
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0.0

def pairwise_jaccard_similarity(groups):
    """
    Compute the pairwise Jaccard similarity between multiple sets.
    :param sets: List of sets.
    :return: Pairwise Jaccard similarity matrix.
    """
    sets = [set(i) for i in groups]
    n_sets = len(sets)
    
    # Create a matrix to store pairwise Jaccard similarity values
    jaccard_matrix = [[0.0] * n_sets for _ in range(n_sets)]
    
    # Compute pairwise Jaccard similarity
    for i, j in combinations(range(n_sets), 2):
        jaccard = jaccard_similarity(sets[i], sets[j])
        jaccard_matrix[i][j] = jaccard
        jaccard_matrix[j][i] = jaccard
    
    return jaccard_matrix

def global_jaccard_similarity(pairwise_matrix):
    """
    Compute a global Jaccard similarity measure from a pairwise Jaccard similarity matrix.
    :param pairwise_matrix: Pairwise Jaccard similarity matrix.
    :return: Global Jaccard similarity value.
    """
    # Convert the pairwise matrix to a numpy array for easier manipulation
    np_matrix = np.array(pairwise_matrix)

    # Exclude diagonal elements
    np.fill_diagonal(np_matrix, np.nan)

    # Compute the average Jaccard similarity excluding diagonal elements
    average_similarity = np.nanmean(np_matrix)

    return average_similarity

### Example Distributions

In [21]:
# group_dist_max_simiarity = [[0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3]]
# group_dist_min_simiarity = [[0], [1], [2], [3]]
# group_dist_median_simiarity = [[0, 1], [0, 2, 3], [1, 2], [3]]

In [22]:
# Rand index 
# result_rand_index = rand_index(group_dist_max_simiarity)
# print("Rand Index:", result_rand_index)

In [23]:
# Jaccard Index 
# pairwise_jaccard_matrix = pairwise_jaccard_similarity(group_dist_median_simiarity)
# print("Pairwise Jaccard Similarity Matrix:")
# for row in pairwise_jaccard_matrix:
#     print(row)

# global_similarity = global_jaccard_similarity(pairwise_jaccard_matrix)
# print("Global Jaccard Similarity:", global_similarity)

In [24]:
def connection_oriented_cluster_match_joel(groups, contributions):
    # groups: a 2d array. groups[i] is a list of people in group i (assume every person has an index).
    # contributions: an array. contributions[i] is the amount agent i contributed to a project.

    agents = list(range(len(contributions)))

    if any(contributions[i] < 0 for i in agents):
        raise NotImplementedError("negative contributions not supported")

    # memberships[i] is the number of groups agent i is in
    memberships = [len([g for g in groups if i in g]) for i in agents]

    # friend_matrix[i][j] is the number of groups that agent i and j are both in
    friend_matrix = [[len([g for g in groups if i in g and j in g])  for i in agents] for j in agents]

    # build up the funding amount. First, add in everyone's contributions
    funding_amount = sum(contributions)

    def K(i, h):
        if sum([friend_matrix[i][j] for j in h]) > 0:
            return math.sqrt(contributions[i])
        return contributions[i]

    funding_amount += sum(2 * math.sqrt(sum(K(i,p[1])/memberships[i] for i in p[0])) * math.sqrt(sum(K(j,p[0])/memberships[j] for j in p[1])) for p in combinations(groups, 2))
    
    return funding_amount

In [25]:
# result = connection_oriented_cluster_match_joel(group_dist, vote_dist)
# result_sqrt = math.sqrt(result)
# print(result_sqrt)